In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
data = load_wine()
X = data.data
y = data.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)

def create_tf_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, input_dim=X_train.shape[1], activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

class PyTorchModel(nn.Module):
    def __init__(self, input_dim):
        super(PyTorchModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 10)
        self.layer2 = nn.Linear(10, 3)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.softmax(self.layer2(x))
        return x
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
start_time = time.time()
tf_model = create_tf_model()
tf_model.fit(X_train, y_train, epochs=10, verbose=0)
tf_training_time = time.time() - start_time

start_time = time.time()
pytorch_model = PyTorchModel(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pytorch_model.parameters())


C:\Users\kavip\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
pytorch_model.train()
for epoch in range(10):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = pytorch_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
pytorch_training_time = time.time() - start_time

tf_preds = np.argmax(tf_model.predict(X_test), axis=1)
tf_accuracy = accuracy_score(y_test, tf_preds)

pytorch_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = pytorch_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

pytorch_accuracy = correct / total

comparison_table = pd.DataFrame({
    "Framework": ["TensorFlow", "PyTorch"],
    "Training Time (s)": [tf_training_time, pytorch_training_time],
    "Accuracy": [tf_accuracy, pytorch_accuracy]
})

print(comparison_table)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
    Framework  Training Time (s)  Accuracy
0  TensorFlow           3.101619  0.583333
1     PyTorch           7.522035  0.833333
